In [1]:
from set_cover import SetCover
solver = SetCover('test1.txt')

m is 4
n is 10
[10000, 1, 2, 3, 4, 5, 6, 6, 7, 8]
[[1, 8, 4, 7, 9], [8, 6, 9, 1, 2], [1, 3, 2, 4, 9, 5, 6], [1, 3]]


In [2]:
# put timeouts 
# Optimal / Feasible 

In [3]:
solver.solveLP_Rounding()

the LP Rounding scheme takes time 0.0010222739999790065 seconds
the optimal set cover weight is 13


In [4]:
solver.solveILP()

hey1
hey2
hey
